# Лабораторная работа №2
## Выполнил студент группы БФИ2001 Стеклов М. А.

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
3. [Задание 3](#Задание-№3)

### Задание №1
Реализовать методы поиска в соответствии с заданием. Организовать генерацию начального набора случайных данных. Для всех вариантов добавить реализацию добавления, поиска и удаления элементов. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

In [1]:
from random import randint
def get_rand_list(n: int, min_value:int, max_value:int) -> list[int]:
    return [randint(min_value, max_value) for _ in range(n)]

In [2]:
x = get_rand_list(10, -100, 100)
sorted_x = sorted(x)

In [3]:
from abc import ABC, abstractmethod

class SearchClass(ABC):
    def __init__(self, x: list[int]=[]): # x должен быть осортированным списком
        self._x: list[int] = x.copy()
        super().__init__()
            
    @abstractmethod
    def _find_insertion_index(self, el: int) -> int:
        pass
        
    def find(self, el: int) -> int:
        i = self._find_insertion_index(el)
        assert el == self._x[i], "Element not found"
        return i
    
    def insert(self, el: int) -> int:
        index = self._find_insertion_index(el)
        self._x.insert(index, el)
        return index
    
    def remove(self, el: int) -> int:
        i = self.find(el)
        self._x.pop(i)
        return i
    
    def __str__(self) -> str:
        return str(self._x)
        

In [4]:
#Бинарный поиск
class BinarySearch(SearchClass):
    def _find_insertion_index(self, el: int) -> int:
        n = len(self._x)
        l, r = 0, n - 1
        m = (l + r) // 2
        while r - l > 1:
            if el <= self._x[m]:
                r = m
            else:
                l = m
            m = (l + r) // 2
        return r

In [5]:
#Бинарное дерево
from __future__ import annotations

class BinarySearchTree:
    def __init__(self, key: int=None, value: int=None, parent=None):
        self.left: BinarySearchTree = None
        self.right: BinarySearchTree = None
        self.parent: BinarySearchTree = parent
        if key is None or value is None:
            self.key = None
            self.value = None
        else:
            self.key = key
            self.value = value
            
    # Поиск
    def find(self, key: int) -> BinarySearchTree:
        assert key is not None, "Key can't be None"
        assert self.key is not None, f"Element with key {key} not found"
        if key == self.key:
            return self
        return self.left.find(key) if key < self.key else self.right.find(key)
    
    # Вставка
    def insert(self, key: int, value: int):
        assert key is not None, "Key can't be None"
        if self.left is None:
            self.left = BinarySearchTree(parent=self)
        if self.right is None:
            self.right = BinarySearchTree(parent=self)
        if self.key is None or self.key == key:
            self.key = key 
            self.value = value
            return
        if key < self.key:
            self.left.insert(key, value)
        else:
            self.right.insert(key, value)
    
    def find_min(self):
        if self.left.key is None:
            return self
        return self.left.find_min()
    
    # Удаление
    def remove(self, key: int):
        assert key is not None, "Key can't be None"
        assert self.key is not None, f"Element with key {key} not found"
        if key < self.key:
            self.left.remove(key)
        elif key > self. key:
            self.right.remove(key)
        else:
            if self.left.key is None:
                if self.right.key is None:
                    self.key = None
                    self.value = None
                    self.left = None
                    self.right = None
                else:
                    if self.key < self.parent.key:
                        self.parent.left = self.right
                    else:
                        self.parent.right = self.right
            else:
                if self.right.key is None:
                    if self.key < self.parent.key:
                        self.parent.left = self.left
                    else:
                        self.parent.right = self.left
                else:
                    next_node = self.right.find_min()
                    self.key = next_node.key
                    self.value = next_node.value
                    next_node.remove(next_node.key)
    
    def __str__(self) -> str:
        if self.key is None:
            return ''
        str_value = (('' if self.left.key is None else '_')+
                     str(self.key) +
                     ('' if self.right.key is None else '_'))
        if self.left.key is None and self.right.key is None:
            return str_value
        split_left = str(self.left).split('\n')
        len_left = len(split_left)
        split_right = str(self.right).split('\n')
        len_right = len(split_right)
        if len_right > len_left:
            split_left += [' ' * len(split_left[0])
                               for _ in range(len_right - len_left)]
        else:
            split_right += [' ' * len(split_right[0])
                                for _ in range(len_left - len_right)]
        value_offset = ' ' * len(str_value)
        str_children = '\n'.join(map(lambda x: x[0] + value_offset + x[1],
                                     zip(split_left, split_right)))
        left_offset = ' ' * len(split_left[0])
        for i in range(len(split_left[0]) - 1, -1, -1):
            if split_left[0][i] not in (' ', '_'):
                left_offset = ' ' * i + '_' * (len(split_left[0]) - i)
                break
        right_offset = ' ' * len(split_right[0])
        for i in range(len(split_right[0])):
            if split_right[0][i] not in (' ', '_'):
                right_offset = '_' * (i + 1) + ' ' * (len(split_right[0]) - i - 1)
                break
        return left_offset + str_value + right_offset + '\n' + str_children
        

In [6]:
tree = BinarySearchTree()
for el in x:
    tree.insert(el, el)
print(tree, end='\n\n')
tree.insert(15, 15)
print(tree, end='\n\n')
print(tree.find(tree.left.key), end='\n\n')
tree.remove(15)
print(tree, end='\n\n')
    


          __-19___________       
  ______-59            __85_____ 
-93__            _____59     __88
    -69         18__        86   
                   41            

          __-19______________       
  ______-59               __85_____ 
-93__               _____59     __88
    -69          __18__        86   
                15    41            

  ______-59
-93__      
    -69    

          __-19___________       
  ______-59            __85_____ 
-93__            _____59     __88
    -69         18__        86   
                   41            



In [20]:
#Метод Фибоначчи
class FibSearch(SearchClass):
    def __init__(self, x: list[int]=[]):
        super().__init__(x=x)
        self._fib_seq = [0, 1]
    def _fib(self, x) -> int:
        if x < len(self._fib_seq):
            return self._fib_seq[x]
        for i in range(x - len(self._fib_seq) + 1):
            self._fib_seq.append(self._fib_seq[-1] + self._fib_seq[-2])
        return self._fib_seq[x]
    def _get_k(self) -> int:
        for i in range(2, len(self._x)):
            if self._fib(i) >= len(self._x) + 1:
                return i - 1
    def _start_init(self):
        self._stop = False
        n = len(self._x)
        k = self._get_k()
        m = self._fib(k + 1) - (n + 1)
        self._i = self._fib(k) - m
        self._p = self._fib(k - 1)
        self._q = self._fib(k - 2)
    def _up_index(self):
        if self._p == 1:
            self._stop = True
        self._i += self._q
        self._p -= self._q
        self._q -= self._p
    def _down_index(self):
        if self._q == 0:
            self._stop = True
        self._i -= self._q
        self._q, self._p = self._p - self._q, self._q
    def _find_insertion_index(self, el: int) -> int:
        self._start_init()
        res_i = -1
        while not self._stop:
            if self._i < 0:
                self._up_index()
            elif self._i >= len(self._x):
                self._down_index()
            elif self._x[self._i] == el:
                return(self._i)
            elif el < self._x[self._i]:
                self._down_index()
            elif el > self._x[self._i]:
                self._up_index()
        return self._i + 1 if self._x[self._i] < el else self._i

In [22]:
bs = BinarySearch(sorted_x)
fs = FibSearch(sorted_x)
print(bs)
print(fs)
bs.insert(15)
fs.insert(15)
print(bs)
print(fs)
bs.remove(15)
fs.remove(15)
print(bs.find(15))

[-93, -69, -59, -19, 18, 41, 59, 85, 86, 88]
[-93, -69, -59, -19, 18, 41, 59, 85, 86, 88]
[-93, -69, -59, -19, 15, 18, 41, 59, 85, 86, 88]
[-93, -69, -59, -19, 15, 18, 41, 59, 85, 86, 88]


AssertionError: Element not found

In [33]:
l = sorted_x.copy()
bs = BinarySearch(sorted_x)
fs = FibSearch(sorted_x)
bst = BinarySearchTree()
for el in l:
    bst.insert(el, el)
print(l) 
from bisect import bisect_right, bisect

[-93, -69, -59, -19, 18, 41, 59, 85, 86, 88]


In [34]:
%%timeit
for i in range(10):
    bs.insert(15)
    bs.find(15)
    bs.remove(15)

28.5 µs ± 225 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [35]:
%%timeit
for i in range(10):
    fs.insert(15)
    fs.find(15)
    fs.remove(15)

158 µs ± 1.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [36]:
%%timeit
for i in range(10):
    bst.insert(15, 15)
    bst.find(15)
    bst.remove(15)

60.5 µs ± 1.52 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [42]:
%%timeit
for i in range(10):
    l.insert(bisect_right(l, 15), 15) 
    bisect_right(l, 15)
    l.pop(bisect_left(l, 15))

5.18 µs ± 145 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [9]:
#Интерполяционный
class InterpolationSearch(SearchClass):
    def _find_insertion_index(self, el: int) -> int:
        x = self._x
        n = len(x)
        l, r = 0, n - 1
        while x[l] < el < x[r]:
            m = l + (el-x[l]) * (r-l) // (x[r]-x[l])
            if x[m] < el:
                l = m + 1
            elif x[m] > el:
                r = m - 1
            else:
                return m
        if x[l] == el:
            return l
        elif x[r] == el: 
            return r
        else:
            return r + 1 if x[r] - el < el - x[l] else l

In [10]:
a = InterpolationSearch(sorted_x)
print(sorted_x)
a.insert(1)
print(a)
a.remove(1)
print(a.find(1))
print(a)

[-93, -69, -59, -19, 18, 41, 59, 85, 86, 88]
[-93, -69, -59, -19, 1, 18, 41, 59, 85, 86, 88]


AssertionError: Element not found

In [45]:
ins = InterpolationSearch(sorted_x)

In [46]:
%%timeit
for i in range(10):
    ins.insert(15)
    ins.find(15)
    ins.remove(15)

36.3 µs ± 1.56 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Задание №2
Написать соответствующие решения для заданных методов.

In [11]:
from math import sqrt
#Простое рехэширование
class HashTable(ABC):
    def __init__(self, n: int=3):
        self._n = n
        self._size = 2**n
        self._const = int(self._size*(sqrt(5) - 1)/2)
        self._table = [None] * self._size
        self._values_count = 0
        super().__init__()
    
    @abstractmethod
    def insert(self, key: int, value: int):
        pass
    
    @abstractmethod
    def find(self, key: int):
        pass
    
    @abstractmethod
    def remove(self, key: int):
        pass
    
                
    def _get_hash(self, key):
        s = 32 - self._n
        return (key * 2654435769 % 4294967296) >> s
    
    def __str__(self) -> str:
        return '\n'.join(map(str, self._table))

In [12]:
class SimpleAndRandomHashTable(HashTable):
    def __init__(self, n=3):
        super().__init__(n=n)
        self._indexes = self._get_indexes()
        
    def insert(self, key: int, value: int):
        hash_key = self._get_hash(key)
        if self._table[hash_key] is None:
            self._table[hash_key] = (key, value)
        elif self._table[hash_key][0] != key:
            i = self._solve_collision(key, hash_key)
            self._table[i] = (key, value)
        else:
            self._table[hash_key] = (key, value)
            return
        self._values_count += 1
        if self._values_count >= self._size // 2:
            self._double_table_size()
            
    def find(self, key: int):
        hash_key = self._get_hash(key)
        assert self._table[hash_key] is not None, f"Element with key {key} not found"
        if self._table[hash_key][0] == key:
            return self._table[hash_key][1]
        return self._find_with_collision(key, hash_key)
    
    def remove(self, key: int):
        hash_key = self._get_hash(key)
        last_index = None
        for i in [0] + self._indexes:
            index = (hash_key + i) % self._size
            if self._table[index] is None:
                break
            if self._table[index][0] == key:
                if last_index is None:
                    self._table[index] = None
                else:
                    self._table[index], self._table[last_index] = (
                    self._table[last_index], self._table[index])
                last_index = index
        assert last_index is not None, f"Element with key {key} not found"
    
    def _double_table_size(self):
        self._n += 1
        self._size *= 2
        self._indexes = self._get_indexes()
        self._const = int(self._size*(sqrt(5) - 1)/2)
        self._table, old_table = [None] * self._size, self._table[::]
        self._values_count, values_left = 0, self._values_count
        for el in old_table:
            if el is not None:
                self.insert(el[0], el[1])
                values_left -= 1
            if values_left == 0:
                break
    
    def _solve_collision(self, key: int, hash_key: int):
        for i in self._indexes:
            index = (hash_key + i) % self._size
            if self._table[index] is None:
                return index
        
    def _find_with_collision(self, key: int, hash_key: int):
        for i in self._indexes:
            index = (hash_key + i) % self._size
            assert self._table[index][0] is not None, f"Element with key {key} not found"
            if self._table[index][0] == key:
                return self._table[i][1]
    
    @abstractmethod
    def _get_indexes(self):
        pass


In [13]:
class SimpleHashTable(SimpleAndRandomHashTable):
    def _get_indexes(self):
        return range(1, self._size)

In [14]:
#Рехэширование с помощью псевдослучайных чисел
from random import shuffle
class RandomHashTable(SimpleAndRandomHashTable):
    def _get_indexes(self):
        l = list(range(1, self._size))
        shuffle(l)
        return l

In [15]:
#Метод цепочек

In [16]:
class ChainsHashTable(HashTable):
    def insert(self, key: int, value: int):
        hash_key = self._get_hash(key)
        if self._table[hash_key] is None:
            self._table[hash_key] = [(key, value)]
        else:
            for i in range(len(self._table[hash_key])):
                if self._table[hash_key][i][0] == key:
                    self._table[hash_key][i] = (key, value)
                    return 
            self._table[hash_key].append((key, value))
        self._values_count += 1
        if self._values_count >= self._size // 2:
            self._double_table_size()
    
    def find(self, key: int):
        hash_key = self._get_hash(key)
        assert self._table[hash_key] is not None, f"Element with key {key} not found"
        for el in self._table[hash_key]:
            if el[0] == key:
                return el[1]
        assert False, f"Element with key {key} not found"
    
    def remove(self, key: int):
        hash_key = self._get_hash(key)
        assert self._table[hash_key] is not None, f"Element with key {key} not found"
        for i in range(len(self._table[hash_key])):
            if self._table[hash_key][i][0] == key:
                self._table[hash_key].pop(i)
                if len(self._table[hash_key]) == 0:
                    self._table[hash_key] = None
                return
        assert False, f"Element with key {key} not found"
    
    def _double_table_size(self):
        self._n += 1
        self._size *= 2
        self._const = int(self._size*(sqrt(5) - 1)/2)
        self._table, old_table = [None] * self._size, self._table[::]
        self._values_count, values_left = 0, self._values_count
        for l in old_table:
            if l is not None:
                for el in l:
                    self.insert(el[0], el[1])
                    values_left -= 1
            if values_left == 0:
                break
                

In [17]:
a = ChainsHashTable()
for el in x:
    a.insert(el, el)
b = SimpleHashTable()
for el in x:
    b.insert(el, el)
c = RandomHashTable()
for el in x:
    c.insert(el, el)
print(a, end='\n\n')
print(b, end='\n\n')
print(c, end='\n\n')

None
None
None
[(18, 18)]
[(86, 86)]
None
None
None
[(-19, -19)]
None
[(41, 41)]
[(-69, -69)]
[(88, 88)]
None
[(59, 59)]
None
[(-93, -93)]
[(85, 85), (-59, -59)]
None
None
None
None
None
None
None
None
None
None
None
None
None
None

None
None
None
(18, 18)
(86, 86)
None
None
None
(-19, -19)
None
(41, 41)
(-69, -69)
(88, 88)
None
(59, 59)
None
(-93, -93)
(85, 85)
(-59, -59)
None
None
None
None
None
None
None
None
None
None
None
None
None

None
None
None
(18, 18)
(86, 86)
None
None
None
(-19, -19)
None
(41, 41)
(-69, -69)
(88, 88)
None
(59, 59)
None
(-93, -93)
(85, 85)
None
None
None
None
None
None
None
None
None
None
None
None
(-59, -59)
None



### Задание №3
Расставить на стандартной 64-клеточной шахматной доске 8 ферзей так, чтобы ни один из них не находился под боем другого». Подразумевается, что ферзь бьёт все клетки, расположенные по вертикалям, горизонталям и обеим диагоналям
Написать программу,  которая находит хотя бы один способ решения задач.


In [18]:
def helper(board: set[tuple[int, int]], depth=0) -> tuple[bool, list[tuple[int, int]]]:
    if depth == 8:
        return (True, [])
    if len(board) == 0:
        return (False, [])
    my_board = board.copy()
    for i, j in board:
        my_board -= {(i, _j) for _j in range(8)}
        my_board -= {(_i, j) for _i in range(8)}
        d = i - j
        i_s, j_s = (0, -d) if d <= 0 else (d, 0)
        my_board -= {(i_s + s, j_s + s) for s in range(min(8 - i_s, 8 - j_s))}
        d = 7 - j - i
        i_s, j_s = (-d, 7) if d <= 0 else (0, 7 - d)
        my_board -= {(i_s + s, j_s - s) for s in range(min(8 - i_s, j_s))}
        if (res := helper(my_board, depth=depth + 1))[0]:
            res[1].append((i, j))
            return res
        my_board = board.copy()
    return (False, [])

from IPython.display import display, Markdown
def task3():
    board = {(i, j) for i in range(8) for j in range(8)}
    board = helper(board)[1]
    out_board = [[(lambda y, x: '▢' if (i + j) % 2 == 0 else '▣')(i, j)
                  for j in range(8)] for i in range(8)]
    for i, j in board:
        out_board[i][j] = '♕' if (i + j) % 2 == 0 else '♛'
    display(Markdown("<br>".join(map(lambda x: f"""<font size="5">{' '.join(x)}</font>""", out_board))))
task3()

<font size="5">▢ ▣ ♕ ▣ ▢ ▣ ▢ ▣</font><br><font size="5">▣ ▢ ▣ ▢ ♛ ▢ ▣ ▢</font><br><font size="5">▢ ♛ ▢ ▣ ▢ ▣ ▢ ▣</font><br><font size="5">▣ ▢ ▣ ▢ ▣ ▢ ▣ ♕</font><br><font size="5">♕ ▣ ▢ ▣ ▢ ▣ ▢ ▣</font><br><font size="5">▣ ▢ ▣ ▢ ▣ ▢ ♛ ▢</font><br><font size="5">▢ ▣ ▢ ♛ ▢ ▣ ▢ ▣</font><br><font size="5">▣ ▢ ▣ ▢ ▣ ♕ ▣ ▢</font>